In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


# Cargar el dataset
df = pd.read_csv('https://raw.githubusercontent.com/CristopherGR/crime_prediction/main/procesado_de_datos/datos_finales.csv')

# Preprocesamiento de los datos
# Convertir la columna de fecha a datetime
df['fecha_detencion'] = pd.to_datetime(df['fecha_detencion'])

# Crear una nueva columna para el año
df['anio'] = df['fecha_detencion'].dt.year

# Filtrar los datos para incluir solo hasta 2023
df = df[df['anio'] <= 2023]

# Codificar las variables categóricas
le_provincia = LabelEncoder()
df['provincia'] = le_provincia.fit_transform(df['provincia'])

le_presunta_subinfraccion = LabelEncoder()
df['presunta_subinfraccion'] = le_presunta_subinfraccion.fit_transform(df['presunta_subinfraccion'])

le_sexo = LabelEncoder()
df['sexo'] = le_sexo.fit_transform(df['sexo'])

le_nacionalidad = LabelEncoder()
df['nacionalidad'] = le_nacionalidad.fit_transform(df['nacionalidad'])

# Crear un nuevo dataset con la cantidad de crímenes por provincia y año
df_crimenes = df.groupby(['provincia', 'anio']).size().reset_index(name='cantidad_crimenes')

# Crear características y objetivo
X = df_crimenes.drop('cantidad_crimenes', axis=1)
y = df_crimenes['cantidad_crimenes']

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo de Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'METRICAS DE DESEMPEÑO PARA RANDOM FOREST')
print(f'   >     MAPE: {mape}%')
print(f'   >     MAE: {mae}')
print(f'   >     R²: {r2}')

# 9. Hacer predicciones para el año 2024 y 2025
future_years = [2024, 2025]
future_data = []

for year in future_years:
    for province in range(len(le_provincia.classes_)):
        future_data.append([province, year])

future_df = pd.DataFrame(future_data, columns=['provincia', 'anio'])

# Predicciones
future_predictions = model.predict(future_df)

# Convertir las predicciones a un DataFrame
future_predictions_df = future_df.copy()
future_predictions_df['prediccion_crimenes'] = future_predictions

# Decodificar las provincias
future_predictions_df['provincia'] = le_provincia.inverse_transform(future_predictions_df['provincia'])

# Mostrar las predicciones
future_predictions_df = future_predictions_df.sort_values(by='provincia').reset_index(drop=True)
future_predictions_df

METRICAS DE DESEMPEÑO PARA RANDOM FOREST
   >     MAPE: 33.36990014497485%
   >     MAE: 724.9395833333332
   >     R²: 0.9419141901744479


,provincia,anio,prediccion_crimenes
0,AZUAY,2024,2740.20
1,AZUAY,2025,2740.20
2,BOLIVAR,2025,816.72
3,BOLIVAR,2024,816.72
4,CARCHI,2024,1587.33
5,CARCHI,2025,1587.33
6,CAÑAR,2024,1186.45
7,CAÑAR,2025,1186.45
8,CHIMBORAZO,2024,1990.85
9,CHIMBORAZO,2025,1990.85
